In [2]:
import cv2
import tensorflow as tf
from PIL import Image
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tkinter as tk
from fastai.vision import *

/Users/jamesrogers/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# URL of the model we want to use
model_url = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2"

# Load the model
model = hub.load(model_url)


In [4]:
from fastai.vision.all import load_learner
from fastai.vision.all import*

# Load the trained model
learner = load_learner('card_classifier_update.pkl')


In [5]:
categories=('Ace of clubs','Ace of diamonds','ace of hearts','ace of spades','eight of clubs','eight of diamonds','eight of hearts','eight of spades','five of clubs','five of diamonds','five of hearts', 'five of spades','four of clubs','four of diamonds','four of hearts','four of spades','jack of clubs','jack of diamonds','jack of hearts','jack of spades','joker','king of clubs','king of diamonds','king of hearts','king of spades','nine of clubs','nine of diamonds','nine of hearts','nine of spades','queen of clubs','queen of diamonds','queen of hearts','queen of spades','seven of clubs','seven of diamonds','seven of hearts','seven of spades','six of clubs','six of diamonds','six of hearts','six of spades','ten of clubs','ten of diamonds','ten of hearts','ten of spades','three of clubs','three of diamonds','threeof hearts',' three of spades',' two of clubs', 'two of diamonds',' two of hearts','two of spades')

def classify_image(img):
    img = img.resize((224, 224))  # Resize the image
    pred, idx, probs = learner.predict(img)
    return pred

def card_value(card):
    if any(substring in card for substring in ['two','three','four','five','six']):
        print(f"Card: {card} Count change: 1")
        return 1
    elif any(substring in card for substring in ['seven','eight','nine']):
        print(f"Card: {card} Count change: 0")
        return 0
    else:
        print(f"Card: {card} Count change: -1")
        return -1


### Only counts if card != previous card

In [6]:

# Access the webcam
cap = cv2.VideoCapture(0)

count = 0
last_prediction_time = time.time()

# To keep track of the last card that was detected
last_card = None


while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the image to the format expected by the model
    input_tensor = tf.convert_to_tensor(frame)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Run the model
    output_dict = model(input_tensor)

    # The output is a dict with keys 'detection_scores', 'detection_classes', and 'detection_boxes'
    # Get the number of objects detected
    num_detections = int(output_dict.pop('num_detections'))

    # For each detected object
    for i in range(num_detections):
        # Only consider detections with a high confidence score
        if output_dict['detection_scores'][0][i] > 0.20:
            # Get the bounding box of the detected object
            ymin, xmin, ymax, xmax = output_dict['detection_boxes'][0][i].numpy()
            ymin = int(ymin * frame.shape[0])
            xmin = int(xmin * frame.shape[1])
            ymax = int(ymax * frame.shape[0])
            xmax = int(xmax * frame.shape[1])

            # Draw a rectangle around the detected object
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

            # Crop the detected object from the frame
            cropped_img = frame[ymin:ymax, xmin:xmax]
            cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)
            cropped_img = Image.fromarray(cropped_img)

            # Classify the card and update the count if enough time has passed
            if time.time() - last_prediction_time > 2:  # Adjust the time as needed
                card = classify_image(cropped_img)
                
                # Only update the count and last_card if a new card is detected
                if card != last_card:
                    print(f"Detected card: {card}")  # Print detected card
                    count_change = card_value(card)
                    count += count_change
                    last_card = card  # Update last_card

                last_prediction_time = time.time()

    # Display the count on the video feed
    cv2.putText(frame, f"Count: {count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Display the resulting frame
    cv2.imshow('Frame', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop release the cap object and destroy all windows
cap.release()
cv2.destroyAllWindows()


2024-04-25 03:17:46.827 Python[15033:102134] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Detected card: ace of hearts
Card: ace of hearts Count change: -1


2024-04-25 03:17:52.013 Python[15033:102134] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


Detected card: two of hearts
Card: two of hearts Count change: 1


Detected card: nine of spades
Card: nine of spades Count change: 0


...

Detected card: three of hearts
Card: three of hearts Count change: 1


Detected card: ace of hearts
Card: ace of hearts Count change: -1


Detected card: four of hearts
Card: four of hearts Count change: 1


Detected card: ace of hearts
Card: ace of hearts Count change: -1


: 

### Only counts if new card != any previous card

In [ ]:

# Access the webcam
cap = cv2.VideoCapture(0)

count = 0
last_prediction_time = time.time()

# Set to keep track of all the cards that have been detected
seen_cards = set()

# For loop until 'q' key is pressed
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the image to the format expected by the model
    input_tensor = tf.convert_to_tensor(frame)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Run the model
    output_dict = model(input_tensor)

    # The output is a dict with keys 'detection_scores', 'detection_classes', and 'detection_boxes'
    # Get the number of objects detected
    num_detections = int(output_dict.pop('num_detections'))

    # For each detected object
    for i in range(num_detections):
        # Only consider detections with a high confidence score
        if output_dict['detection_scores'][0][i] > 0.21:
            # Get the bounding box of the detected object
            ymin, xmin, ymax, xmax = output_dict['detection_boxes'][0][i].numpy()
            ymin = int(ymin * frame.shape[0])
            xmin = int(xmin * frame.shape[1])
            ymax = int(ymax * frame.shape[0])
            xmax = int(xmax * frame.shape[1])

            # Draw a rectangle around the detected object
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

            # Crop the detected object from the frame
            cropped_img = frame[ymin:ymax, xmin:xmax]
            cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)
            cropped_img = Image.fromarray(cropped_img)

            # Classify the card and update the count if enough time has passed
            if time.time() - last_prediction_time > 1.5:  # Adjust the time as needed
                card = classify_image(cropped_img)
                
                # Only update the count and last_card if the card is new
                if card not in seen_cards:
                    print(f"Detected card: {card}")  # Print detected card
                    count_change = card_value(card)
                    count += count_change
                    seen_cards.add(card)  # Add card to the set of seen cards

                last_prediction_time = time.time()

    # Display the count on the video feed
    cv2.putText(frame, f"Count: {count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Display the resulting frame
    cv2.imshow('Frame', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop release the cap object and destroy all windows
cap.release()
cv2.destroyAllWindows()


In [7]:
import time

# Access the webcam
cap = cv2.VideoCapture(0)
count = 0
last_prediction_time = time.time()
last_card = None  # To keep track of the last card that was detected

start_time = time.time()  # Record the start time

while time.time() - start_time < 30:  # Run for 30 seconds
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the image to the format expected by the model
    input_tensor = tf.convert_to_tensor(frame)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Run the model
    output_dict = model(input_tensor)

    # The output is a dict with keys 'detection_scores', 'detection_classes', and 'detection_boxes'
    # Get the number of objects detected
    num_detections = int(output_dict.pop('num_detections'))

    # For each detected object
    for i in range(num_detections):
        # Only consider detections with a high confidence score
        if output_dict['detection_scores'][0][i] > 0.20:
            # Get the bounding box of the detected object
            ymin, xmin, ymax, xmax = output_dict['detection_boxes'][0][i].numpy()
            ymin = int(ymin * frame.shape[0])
            xmin = int(xmin * frame.shape[1])
            ymax = int(ymax * frame.shape[0])
            xmax = int(xmax * frame.shape[1])

            # Draw a rectangle around the detected object
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

            # Crop the detected object from the frame
            cropped_img = frame[ymin:ymax, xmin:xmax]
            cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)
            cropped_img = Image.fromarray(cropped_img)

            # Classify the card and update the count if enough time has passed
            if time.time() - last_prediction_time > 2:  # Adjust the time as needed
                card = classify_image(cropped_img)

                # Only update the count and last_card if a new card is detected
                if card != last_card:
                    print(f"Detected card: {card}")  # Print detected card
                    count_change = card_value(card)
                    count += count_change
                    last_card = card  # Update last_card

                last_prediction_time = time.time()

    # Display the count on the video feed
    cv2.putText(frame, f"Count: {count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow('Frame', frame)

    # Break the loop on 'q' key press or if 30 seconds have passed
    if cv2.waitKey(1) & 0xFF == ord('q') or time.time() - start_time >= 30:
        break

# After the loop release the cap object and destroy all windows
cap.release()
cv2.destroyAllWindows()

Detected card: four of diamonds
Card: four of diamonds Count change: 1


Detected card: four of hearts
Card: four of hearts Count change: 1


Detected card: three of diamonds
Card: three of diamonds Count change: 1


Detected card: four of clubs
Card: four of clubs Count change: 1


Detected card: ace of hearts
Card: ace of hearts Count change: -1


Detected card: four of hearts
Card: four of hearts Count change: 1


Detected card: ace of hearts
Card: ace of hearts Count change: -1


Detected card: three of diamonds
Card: three of diamonds Count change: 1
